In [6]:
from random import randint

for i in range(1,11):
    print(randint(1,10))

1
5
2
3
5
3
9
2
8
8


In [13]:
pc_ref = [0, 25, -60, 400, -20, -75, 0, 300, -62.5, 66.7, 0, 40, -42.9, -25, -66.7, 0, 0, 700, -12.5, -28.6]
pc_tst = [0, 400, -60, 400, -20, -75, 0, 0, 300, 0]

def get_chunks(lst):
    r=[]
    chunk_size = 3
    tot = len(lst)
    for idx, val in enumerate(lst):
        end_idx = idx+chunk_size
        if end_idx <= tot:
            r.append(lst[idx:end_idx])
    return r

ref_chunks = get_chunks(pc_ref)
tst_chunks = get_chunks(pc_tst)

In [14]:
print(ref_chunks)
print(tst_chunks)

[[0, 25, -60], [25, -60, 400], [-60, 400, -20], [400, -20, -75], [-20, -75, 0], [-75, 0, 300], [0, 300, -62.5], [300, -62.5, 66.7], [-62.5, 66.7, 0], [66.7, 0, 40], [0, 40, -42.9], [40, -42.9, -25], [-42.9, -25, -66.7], [-25, -66.7, 0], [-66.7, 0, 0], [0, 0, 700], [0, 700, -12.5], [700, -12.5, -28.6]]
[[0, 400, -60], [400, -60, 400], [-60, 400, -20], [400, -20, -75], [-20, -75, 0], [-75, 0, 0], [0, 0, 300], [0, 300, 0]]


In [15]:
def chunk_diff(ref_lst, tst_lst):
    cntr = 0
    chunk_size = len(tst_lst[0])
    sm_lst = []
    sm_idx_dic = {}
    print(chunk_size)
    for i in tst_lst:
        for j in ref_lst:
            diff = []
            for k in range(chunk_size):
                diff.append(abs(i[k] - j[k]))
            cntr += 1
            sm = sum(diff)
            sm_lst.append({'k': f'{cntr}: {i} vs {j}', 'sm': sm})
    return sm_lst

sm_lst = chunk_diff(ref_chunks, tst_chunks)

3


In [16]:
from typing import List, Dict

def sort_dicts_by_value(dicts: List[Dict], key: str) -> List[Dict]:
    """
    Sorts a list of dictionaries by the specified key value.

    :param dicts: List of dictionaries to be sorted
    :param key: The key by which the dictionaries should be sorted
    :return: A list of dictionaries sorted by the specified key
    """
    try:
        return sorted(dicts, key=lambda x: x[key])
    except KeyError:
        raise KeyError(f"One or more dictionaries do not have the key '{key}'")
    except TypeError:
        raise TypeError(f"The value for key '{key}' is not comparable")

sorted_list = sort_dicts_by_value(sm_lst, 'sm')
print(sorted_list)


[{'k': '39: [-60, 400, -20] vs [-60, 400, -20]', 'sm': 0}, {'k': '58: [400, -20, -75] vs [400, -20, -75]', 'sm': 0}, {'k': '77: [-20, -75, 0] vs [-20, -75, 0]', 'sm': 0}, {'k': '105: [-75, 0, 0] vs [-66.7, 0, 0]', 'sm': 8.299999999999997}, {'k': '86: [-20, -75, 0] vs [-25, -66.7, 0]', 'sm': 13.299999999999997}, {'k': '133: [0, 300, 0] vs [0, 300, -62.5]', 'sm': 62.5}, {'k': '114: [0, 0, 300] vs [-75, 0, 300]', 'sm': 75}, {'k': '99: [-75, 0, 0] vs [-62.5, 66.7, 0]', 'sm': 79.2}, {'k': '3: [0, 400, -60] vs [-60, 400, -20]', 'sm': 100}, {'k': '7: [0, 400, -60] vs [0, 300, -62.5]', 'sm': 102.5}, {'k': '104: [-75, 0, 0] vs [-25, -66.7, 0]', 'sm': 116.7}, {'k': '84: [-20, -75, 0] vs [40, -42.9, -25]', 'sm': 117.1}, {'k': '87: [-20, -75, 0] vs [-66.7, 0, 0]', 'sm': 121.7}, {'k': '103: [-75, 0, 0] vs [-42.9, -25, -66.7]', 'sm': 123.80000000000001}, {'k': '95: [-75, 0, 0] vs [-20, -75, 0]', 'sm': 130}, {'k': '85: [-20, -75, 0] vs [-42.9, -25, -66.7]', 'sm': 139.60000000000002}, {'k': '101: [-75

In [17]:

from binance.client import Client
from binance.enums import *
from time import time
import pickle as pickle
from datetime import datetime
import pandas as pd
from pathlib import Path
from datetime import datetime
import json


downloads_path = str(Path.home() / "Downloads")
documents_path = str(Path.home() / "Documents")
tst_fn = f'{downloads_path}\\hd.csv'


def get_client():
    fn = f'{documents_path}\\key\\binance-key.pickle'
    with open(fn, 'rb') as handle:
        k = pickle.load(handle)
    return Client(k['API_KEY'], k['API_SECRET'])

def get_config(file_path):
    with open(file_path, 'r') as file:
        config = json.load(file)
    return config


def get_chunks(lst, chunk_size):
    r=[]
    tot = len(lst)
    for idx, val in enumerate(lst):
        end_idx = idx+chunk_size
        if end_idx <= tot:
            r.append(lst[idx:end_idx])
    return r

def get_pc(hd_df):
    close_price = list(hd_df['close'])
    ts = list(hd_df['time'])
    pc = []
    for idx, val in enumerate(close_price):
        if idx==0:
            pc.append({'time': ts[0], 'pc': 0})
        else:
            previous_price = close_price[idx-1]
            current_price = val
            pc_val = (round((current_price*100/previous_price)-100, 4))
            pc.append({'time': ts[idx], 'pc': pc_val})
    pc_df = pd.DataFrame(pc)
    return pc_df

def get_trades(hd_df, chunk_size):
    close_price = list(hd_df['close'])
    high_price = list(hd_df['high'])
    low_price = list(hd_df['low'])
    longs=[]
    shorts=[]
    long_qualified = []
    short_qualified = []
    tot = len(close_price)
    for idx in range(len(close_price)):
        if (idx + chunk_size) <= tot:
            cp = close_price[idx]
            max_price = max(high_price[idx:idx + chunk_size])
            min_price = min(low_price[idx:idx + chunk_size])
            long_roi = round((max_price*100/cp)-100, 4)
            short_roi = round((cp*100/min_price)-100, 4)
            longs.append({'long': long_roi})
            long_qualified.append(long_roi >= roi_threshold)
            shorts.append({'short': short_roi})
            short_qualified.append(short_roi >= roi_threshold)
        else:
            long_roi = 0.0
            short_roi = 0.0
            longs.append({'long': long_roi})
            long_qualified.append(False)
            shorts.append({'short': short_roi})
            short_qualified.append(False)
    long_trades_df = pd.DataFrame(longs)
    short_trades_df = pd.DataFrame(shorts)
    long_qualified_df = pd.DataFrame(long_qualified)
    short_qualified_df = pd.DataFrame(short_qualified)
    return long_trades_df, short_trades_df, long_qualified, short_qualified

def get_historical_data(start_timestamp, end_timestamp, candlestick): 
    # client = get_client()
    # data = []
    # tot = (end_timestamp - start_timestamp)/(900*500)
    # cntr = 0
    # for current_sts in range(start_timestamp, end_timestamp+1, 900*500):
    #     next_ets = current_sts + 900*500 if (current_sts + 900*500) < end_timestamp else end_timestamp
    #     print(current_sts, next_ets, f'100% completed') if next_ets == end_timestamp else print(current_sts, next_ets, f'{round(cntr*100/tot, 1)}% completed')
    #     cntr += 1
        
    #     klines = client.futures_historical_klines('BTCUSDT', candlestick, current_sts*1000, next_ets*1000, limit=500)
        
    #     for kline in klines:
    #         timestamp = kline[0]/1000
    #         open_price = float(kline[1])
    #         high_price = float(kline[2])
    #         low_price = float(kline[3])
    #         close_price = float(kline[4])
    #         volume = float(kline[5])

    #         data.append([timestamp, open_price, high_price, low_price, close_price, volume])

    # df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    df = pd.read_csv(f'{downloads_path}/hd.csv')
    close_pc_df = get_pc(df)
    long_trades_df, short_trades_df, long_qualified, short_qualified = get_trades(df, chunk_size)
    df['close_pc'] = close_pc_df['pc']
    df['long'] = long_trades_df
    df['short'] = short_trades_df
    df['long_qualified'] = long_qualified
    df['short_qualified'] = short_qualified
    df.to_csv(tst_fn, index=False)
    print('Data Exported')
    return df


config = get_config('config.json')
candlestick = config['candlestick']
chunk_size = config['chunk_size']
roi_threshold = config['roi_threshold']
sm_threshold = config['sm_threshold']

start_timestamp = 1609459200 #  January 1, 2021 12:00:00 AM
end_timestamp = 1719721304
hd_df = get_historical_data(start_timestamp, end_timestamp, candlestick)

ref_chunks = get_chunks(pc_ref)
tst_chunks = get_chunks(pc_tst)


Data Exported


In [19]:
hd_df

,time,open,high,low,close,volume,close_pc,long,short,long_qualified,short_qualified
0,1.609459e+09,28948.19,29045.93,28706.00,28786.75,3181.960,0.0000,2.6390,0.2813,True,False
1,1.609460e+09,28786.92,28902.68,28752.30,28859.28,2215.315,0.2520,2.3810,0.3721,True,False
2,1.609461e+09,28860.00,28968.49,28859.99,28947.61,1153.678,0.3061,2.0686,0.3036,True,False
3,1.609462e+09,28947.61,29055.00,28910.94,29015.00,1486.635,0.2328,1.8315,0.3599,True,False
4,1.609463e+09,29015.00,29499.00,28975.46,29446.35,7476.067,1.4866,0.3398,1.6251,False,True
...,...,...,...,...,...,...,...,...,...,...,...
122509,1.719717e+09,60827.70,60862.40,60777.50,60832.30,490.053,0.0077,0.0000,0.0000,False,False
122510,1.719718e+09,60832.30,60937.20,60827.60,60913.90,644.549,0.1341,0.0000,0.0000,False,False
122511,1.719719e+09,60913.80,60943.60,60879.40,60898.20,551.333,-0.0258,0.0000,0.0000,False,False
122512,1.719720e+09,60898.20,60898.20,60844.90,60867.70,436.889,-0.0501,0.0000,0.0000,False,False


In [2]:
raw_df = pd.read_csv(f'{downloads_path}/tst.csv')

In [2]:
hd_df

NameError: name 'hd_df' is not defined

In [5]:
time = list(raw_df['time'])
open = list(raw_df['open'])
high = list(raw_df['high'])
low = list(raw_df['low'])
close = list(raw_df['close'])
volume = list(raw_df['volume'])

b_df = pd.DataFrame()
b_df['time'] = time
b_df['open'] = open
b_df['high'] = high
b_df['low'] = low
b_df['close'] = close
b_df['volume'] = volume

In [6]:
b_df

,time,open,high,low,close,volume
0,1.672531e+09,16537.5,16538.0,16517.4,16517.4,1519.880
1,1.672532e+09,16517.4,16540.9,16513.4,16539.4,1312.854
2,1.672533e+09,16539.3,16539.4,16504.0,16514.4,1806.819
3,1.672534e+09,16514.5,16533.1,16513.9,16527.0,741.846
4,1.672535e+09,16527.1,16539.0,16524.1,16537.3,935.879
...,...,...,...,...,...,...
52429,1.719717e+09,60827.7,60862.4,60777.5,60832.3,490.053
52430,1.719718e+09,60832.3,60937.2,60827.6,60913.9,644.549
52431,1.719719e+09,60913.8,60943.6,60879.4,60898.2,551.333
52432,1.719720e+09,60898.2,60898.2,60844.9,60867.7,436.889
